# Code to Prototype Uploading Fraudulent Cases to to the Mongo DB

In [ ]:
import pandas as pd
import numpy as np
import junoutils
import junoml
import junodb
import datetime

In [ ]:
# set up the database client

c = junodb.client

p = c['production']

bl = p['emailBlacklistCollection']

In [ ]:
s = pd.read_csv("data/small_summary.csv")
x = s[s.email.str.contains('einstein.exchange') == False]
x = x[x.email.str.contains('@mailinator.com') == False]
x = x[x.email.str.contains('testing@invalidEmail.exchange') == False]
x = x[x.email.str.contains('@mailionator.com') == False]
x = x[x.email.str.contains('@einstine.exchange.com') == False]
x = x[x.email.str.contains('@einstien.exchange.com') == False]
x = x[x.email.str.contains('@eistein.exchange') == False]
x = x[x.email.str.contains('@einstein.exchang') == False]
x = x[x.email.str.contains('@einstein.wxchange') == False]
x = x[x.email.str.contains('@einstein.exchane') == False]
x = x[x.email.str.contains('@einstein.exchan') == False]
x = x[x.email.str.contains('@einstein.exchnage') == False]
x = x[x.email.str.contains('@einsteinexchange.com') == False]
x = x[x.email.str.contains('@fingerfoodstudios.com') == False]
x = x[x.email.str.contains('@finger') == False]
x = x[x.email.str.contains('trevordasilva') == False]
x = x[x.email.isin(['tyson@einsteincapital.ca', 'tysonpower@gmail.com', 'bioviral@gmail.com','testing@einsteain.exchange','themarcellfraser@gmail.com']) == False]
x.sort_values(by='anomaly_score_autoencoder', ascending=False)
s = x.copy()

In [ ]:
numerical = s[list(set(s.columns) - set(junoutils.prefixColumns(['tSNE','mean','median','std_','email','fraud','anomaly'], columnlist=s.columns)))]

In [ ]:
numerical_scaled = junoutils.scaleDf(numerical)

In [ ]:
def insertFraudster(email, source='einsten.exchange', method="MANUAL", level="WARNING", notes=None, created=datetime.datetime.now()):
    
    record = {'email': email, 
              'source': source, 
              'method': method, 
              'level': level, 
              'notes': notes, 
              'created': created}
    
    print(record)
    
    # add logic to insert record  

In [ ]:
for col in sorted(junoutils.prefixColumns(['card','failed','rejected'], numerical.columns)):
    print(col)

In [ ]:
from sklearn.ensemble import IsolationForest
col_list = ['unique_card_name','unique_card_last_digits', 
         'n_cla_customer-vault_verify-card_failed', 
         'n_ca_customer-vault_failed', 'unique_state', 'unique_city']
df = numerical_scaled[col_list]

ifst = IsolationForest(contamination=0.0015)
ifst.fit(df)
preds = ifst.predict(df) == -1
print("Number of Anomalies:", np.sum(ifst.predict(df) == -1))
s[['email','anomaly_score_autoencoder','suspected_fraud']+col_list][preds].sort_values(by='anomaly_score_autoencoder', ascending=False)

In [ ]:
known_fraudsters = s[s.suspected_fraud == True][['email']]
known_fraudsters['source'] = 'einstein.exchange'
known_fraudsters['method'] = 'MANUAL'
known_fraudsters['level'] = 'BLOCKED'
known_fraudsters['notes'] = 'Known fraudulent account flagged manually by Einstein Exchange Staff due to suspicious credit card account activity.'
known_fraudsters['created'] = datetime.datetime.now()
known = known_fraudsters.to_dict(orient='records')
known

In [ ]:
similar_emails = s[s.similar_to_fraud == True][['email']]
similar_emails['source'] = 'einstein.exchange'
similar_emails['method'] = 'AUTOMATIC'
similar_emails['level'] = 'WARNING'
similar_emails['notes'] = 'Emails flagged because they are likely aliases of known fraudulent accounts. Found by running a similarity algorithm.'
similar_emails['created'] = datetime.datetime.now()
similar = similar_emails.to_dict(orient='records')
similar

In [ ]:
for record in known:
    c['production']['emailBlacklistCollection'].insert(record)

In [ ]:
for record in similar:
    c['production']['emailBlacklistCollection'].insert(record)